In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
qq('start')

In [3]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections

In [ ]:
df = pd.read_parquet('df_text.pq')

In [13]:
df.columns

Index(['vacancy_id', 'name', 'specializations', 'key_skills', 'employer',
       'groups_of_spec', 'processed_text', 'key_list', 'name_list'],
      dtype='object')

In [21]:
df.head()

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,key_list,name_list,text_vector
4,2700005,Разработчик Python/django,"[137, 221, 296]",,845fe6266ab1,[1],небольшой дружный команда требоваться опытный ...,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157..."
5,2700006,Торговый представитель,"[149, 242]",работа текущий база клиент мерчендайзинг работ...,23ead4a8f164,[17],обязанность компания дистрибьютор продукт пита...,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080..."
6,2700007,Менеджер по оптовым продажам тканей,"[149, 196, 242, 324, 538]",b2b продажа активный продажа развитие продажа ...,9e3008e9f553,[17],стартекс крупный текстильный компания лидер ры...,"[b2b, продажа, активный, продажа, развитие, пр...","[Менеджер, по, оптовым, продажам, тканей]","[-0.39096445, 0.33778182, -0.016544262, -0.177..."
8,2700010,Врач клинической лабораторной диагностики (Бак...,"[398, 537, 587]",,49f3280f53b6,[13],компания kdl клинико диагностический лаборатор...,[],"[Врач, клинической, лабораторной, диагностики,...","[-0.23469944, 0.35904798, 0.1611339, -0.071664..."
12,2700015,Швея-портной,"[129, 157, 516]",,24e6806caa88,"[18, 11, 29]",требоваться швея лаборант навык закройщик прои...,[],[Швея-портной],"[-0.5796743, 0.25497875, 0.35563952, -0.068753..."


In [5]:
model = Doc2Vec.load('models/vector_size:300_min_count:2_epochs:10_window:7_seed:42_workers:4_negative:50.bin')

In [17]:
def inf_vec(txt):
    return model.infer_vector(txt.split(' '))

In [19]:
df['text_vector'] = pqdm(df['processed_text'], inf_vec, n_jobs=10)

In [20]:
df.to_parquet('df_vector.pq')

In [27]:
def unpivot(obj):
    res = []
    for s in obj.specializations:
        new_obj = obj.to_dict()
        new_obj.pop('specializations')
        new_obj['specialization'] = s
        res.append(new_obj)
    return res

In [4]:
df=pd.read_parquet('df_vector.pq')

In [5]:
df.drop(columns=['processed_text', 'name', 'key_skills', 'groups_of_spec'], inplace = True)

In [6]:
df.head()

,vacancy_id,specializations,employer,key_list,name_list,text_vector
4,2700005,"[137, 221, 296]",845fe6266ab1,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157..."
5,2700006,"[149, 242]",23ead4a8f164,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080..."
6,2700007,"[149, 196, 242, 324, 538]",9e3008e9f553,"[b2b, продажа, активный, продажа, развитие, пр...","[Менеджер, по, оптовым, продажам, тканей]","[-0.39096445, 0.33778182, -0.016544262, -0.177..."
8,2700010,"[398, 537, 587]",49f3280f53b6,[],"[Врач, клинической, лабораторной, диагностики,...","[-0.23469944, 0.35904798, 0.1611339, -0.071664..."
12,2700015,"[129, 157, 516]",24e6806caa88,[],[Швея-портной],"[-0.5796743, 0.25497875, 0.35563952, -0.068753..."


In [ ]:
df.to_parquet('df_vector.pq')

In [9]:
test_vac_id = set(dill.load(open('validatin.bin','rb')))

In [10]:
df['validation'] = df['vacancy_id'].apply(lambda x: x in test_vac_id)

In [33]:
res = []
for i, r in tqdm(df.iterrows(), total = df.shape[0]):
    res += unpivot(r)

In [37]:
df_unpivoted = pd.DataFrame(res)

In [38]:
df_unpivoted.shape

(4491645, 6)

In [39]:
df_unpivoted.to_parquet('df_unp_vector.pq')